In [2]:
import pulp

## Input variables

classes = ["1a", "1b", "1c", "1d", "1e"]

courses = ["English", "German", "Dutch", "Math", "Biology", "Mentorship hour"]

teachers = ["Bob", "Thea", "Johan", "Hanneke", "Kim", "Martin", "Sarah"]

classrooms = ["0.1", "0.2", "1.1", "1.2", "1.3", "2.2", "2.3"]

hours = [1, 2, 3, 4, 5, 6, 7, 8]


## Parameters

# shows for each teacher which courses he/she is able to give
able_to_give_course = [
    [1, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
]

# shows how many hours of each course each class needs
required_hours = [
    [2, 1, 2, 2, 1, 0],
    [2, 2, 2, 1, 1, 0],
    [2, 0, 2, 2, 1, 1],
    [2, 0, 2, 2, 1, 1],
    [2, 0, 2, 2, 1, 1],
]


## Decision variable

X = pulp.LpVariable.dicts(
    "X",
    (
        (h, c, t, r, l)
        for h in hours
        for c in classes
        for t in teachers
        for r in classrooms
        for l in courses
    ),
    cat="Binary",
)
# this variable equals 1 if course is given at hour h, to class c, by teacher t in class room r, else 0.


## objective

prob = pulp.LpProblem("School_Timetable", pulp.LpMaximize)
prob += pulp.lpSum(
    X[h, c, t, r, l]
    for h in hours
    for c in classes
    for t in teachers
    for r in classrooms
    for l in courses
)


## constraints

# No teacher can be double-booked
for h in hours:
    for t in teachers:
        prob += (
            pulp.lpSum(
                X[h, c, t, r, l] for c in classes for r in classrooms for l in courses
            )
            <= 1
        )

# No class can have two lessons at the same time
for h in hours:
    for c in classes:
        prob += (
            pulp.lpSum(
                X[h, c, t, r, l] for t in teachers for r in classrooms for l in courses
            )
            <= 1
        )

# No room can be double-booked
for h in hours:
    for r in classrooms:
        prob += (
            pulp.lpSum(
                X[h, c, t, r, l] for c in classes for t in teachers for l in courses
            )
            <= 1
        )

# Required teaching hours for each class & course
for c_idx, c in enumerate(classes):
    for co_idx, l in enumerate(courses):
        required = required_hours[c_idx][co_idx]
        prob += (
            pulp.lpSum(
                X[h, c, t, r, l] for h in hours for t in teachers for r in classrooms
            )
            <= required
        )

# A teacher can only teach a course if allowed
for t_idx, t in enumerate(teachers):
    for co_idx, l in enumerate(courses):
        if able_to_give_course[t_idx][co_idx] == 0:
            for h in hours:
                for c in classes:
                    for r in classrooms:
                        prob += X[h, c, t, r, l] == 0


## Solving the problem
prob.solve()
print("Status:", pulp.LpStatus[prob.status])

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var/folders/sk/k2jvbc391_d47y527rssnps00000gp/T/7679ecbdd3904df2860d1b331befff04-pulp.lp
 -o /var/folders/sk/k2jvbc391_d47y527rssnps00000gp/T/7679ecbdd3904df2860d1b331befff04-pulp.sol
Reading problem data from '/var/folders/sk/k2jvbc391_d47y527rssnps00000gp/T/7679ecbdd3904df2860d1b331befff04-pulp.lp'...
8582 rows, 11760 columns, 55440 non-zeros
11760 integer variables, all of which are binary
67500 lines were read
GLPK Integer Optimizer 5.0
8582 rows, 11760 columns, 55440 non-zeros
11760 integer variables, all of which are binary
Preprocessing...
177 rows, 2576 columns, 10304 non-zeros
2576 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 177
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
177 rows, 2576 columns, 10304 non-zeros
*    

In [5]:
import pandas as pd

schedule = []

for h in hours:
    for c in classes:
        for t in teachers:
            for r in classrooms:
                for l in courses:
                    if pulp.value(X[h, c, t, r, l]) == 1:
                        schedule.append(
                            {
                                "Hour": h,
                                "Class": c,
                                "Course": l,
                                "Teacher": t,
                                "Room": r,
                            }
                        )

df = pd.DataFrame(schedule)

# show full time table
df = df.sort_values(by=["Class", "Hour"]).reset_index(drop=True)
df
# # show timetable for specific class
# df[df['Class'] == '1a']

# # show timetable for specific teacher
# df[df['Teacher']== 'Bob']

,Hour,Class,Course,Teacher,Room
0,1,1a,Dutch,Sarah,1.2
1,2,1a,English,Bob,0.1
2,3,1a,English,Bob,0.1
3,4,1a,Math,Johan,0.1
4,5,1a,Math,Johan,2.2
5,6,1a,German,Kim,0.1
6,7,1a,Biology,Hanneke,0.2
7,8,1a,Dutch,Sarah,0.1
8,1,1b,Biology,Hanneke,0.2
9,2,1b,German,Kim,1.3
